<a href="https://colab.research.google.com/github/wooihaw/ai_with_python_2024/blob/main/handson_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hands-on 6
#### In this hands-on, an LLM will be installed and served.
#### Retrieval Augmented Generation (RAG) will be used to to produce more accurate and contextually relevant outputs

In [ ]:
!curl https://ollama.ai/install.sh | sh

In [ ]:
!ollama --version

In [ ]:
pip install colab-xterm ollama chromadb

In [ ]:
%load_ext colabxterm
%xterm

In [ ]:
!ollama pull phi3

In [ ]:
!ollama pull mxbai-embed-large

In [ ]:
import ollama

response = ollama.chat(model='phi3', messages=[
  {
    'role': 'user',
    'content': 'Who is Ahmed Bin Careem',
  },
])
print(response['message']['content'])

In [ ]:
import chromadb
from ollama import Client

llm = Client(host="http://localhost:11434")

documents = [
              "Ahmed Bin Careem is a professor at Imaginative University."
              "Ahmed Bin Careem holds a Ph.D. in Artificial Intelligence."
              "Ahmed Bin Careem is a highly qualified and experienced educator with expertise in cutting-edge technologies such as Machine Learning, Deep Learning and Large Language Models.",
]

client = chromadb.Client()
collection = client.create_collection(name="docs")

# store each document in a vector embedding database
for i, d in enumerate(documents):
    response = llm.embeddings(model="mxbai-embed-large", prompt=d)
    embedding = response["embedding"]
    collection.add(ids=[str(i)], embeddings=[embedding], documents=[d])

# an example prompt
prompt = "Who is Ahmed Bin Careem?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = llm.embeddings(prompt=prompt, model="mxbai-embed-large")
results = collection.query(query_embeddings=[response["embedding"]], n_results=1)
data = results["documents"][0][0]

# generate a response combining the prompt and data we retrieved in step 2
output = llm.generate(
    model="phi3", prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output["response"])